In [11]:
# Data loading
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, f1_score
from IPython.display import clear_output

from joblib import load
from tqdm import trange
from tqdm.notebook import tqdm


# Graph dataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset, Data

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedKFold


# GNN Model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv, GATConv, GATv2Conv, SAGEConv


# Sparse vector
from Sparse_vector.sparse_vector import SparseVector

# Data loading

In [12]:
chrom_names = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y','M']]

features = [i[:-4] for i in os.listdir('z_dna/hg38_features/sparse/') if i.endswith('.pkl')]
groups = ['DNase-seq', 'Histone', 'RNA polymerase', 'TFs and others']
feature_names = [i for i in features]

In [13]:
def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'z_dna/hg38_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"z_dna/hg38_dna/{file}") for file in files])

In [21]:
%%time
DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chrom_names)}
#ZDNA = load('z_dna/hg38_zdna/sparse/ZDNA_shin.pkl')
#ZDNA = load('z_dna/hg38_zdna/sparse/ZDNA_cousine.pkl')
#ZDNA = load('z_dna/hg38_zdna/sparse/Kouzine_hg38.pkl')

ZDNA = load('z_dna/hg38_zdna/sparse/EndoQuad_hg19.pkl')

DNA_features = {feature: load(f'z_dna/hg38_features/sparse/{feature}.pkl')
                for feature in tqdm(feature_names)}

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1946 [00:00<?, ?it/s]

CPU times: user 2min 54s, sys: 3.71 s, total: 2min 57s
Wall time: 2min 57s


In [22]:
np.random.seed(10)
width = 100
ints_in = []
ints_out = []

for chrm in chrom_names:
    for st in trange(0, ZDNA[chrm].shape - width, width):
        interval = [st, min(st + width, ZDNA[chrm].shape)]
        if ZDNA[chrm][interval[0]: interval[1]].any():
            ints_in.append([chrm, interval[0], interval[1]])
        else:
            ints_out.append([chrm, interval[0], interval[1]])

ints_in = np.array(ints_in)
ints_out = np.array(ints_out)[np.random.choice(range(len(ints_out)), size=len(ints_in) * 2, replace=False)]

100%|██████████████████████████████████████████████████████████████████████████████| 165/165 [00:00<00:00, 22791.38it/s]


In [19]:
len(ints_in)

45201

In [23]:
len(ints_in)

134686

In [16]:
!nvidia-smi

Sat Sep 14 00:15:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off | 00000000:05:00.0 Off |                  N/A |
| 71%   86C    P2             301W / 250W |   8237MiB / 11264MiB |     88%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Подсчет статистики по интервалам

In [5]:
def find_intervals(vector):
    intervals = []
    start = None

    for i, val in enumerate(vector):
        if val == 1 and start is None:
            start = i
        elif val == 0 and start is not None:
            intervals.append(i - start)
            start = None

    if start is not None:
        intervals.append(len(vector) - start)

    return intervals

# Shin

In [11]:
width_in = 5000
all_intervals = []
for chrm in chrom_names:
    for st in trange(0, ZDNA[chrm].shape - width_in, width_in):
        interval = [st, min(st + width_in, ZDNA[chrm].shape)]
        if ZDNA[chrm][interval[0]: interval[1]].any():
            all_intervals.extend(find_intervals(ZDNA[chrm][interval[0]: interval[1]]))

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 9679.16it/s]


In [12]:
if all_intervals:
    average_length = np.mean(all_intervals)
    min_length = np.min(all_intervals)
    max_length = np.max(all_intervals)
    median_length = np.median(all_intervals)

    print(f'Средний размер интервала: {average_length}')
    print(f'Минимальный размер интервала: {min_length}')
    print(f'Максимальный размер интервала: {max_length}')
    print(f'Медианный размер интервала: {median_length}')
    print(f'Количество интервалов: {len(all_intervals)}')
else:
    print('Интервалы не найдены')

Средний размер интервала: 353.6348039215686
Минимальный размер интервала: 3
Максимальный размер интервала: 551
Медианный размер интервала: 385.0
Количество интервалов: 408


# Кузин

In [82]:
width_in = 5000
all_intervals = []
for chrm in chrom_names:
    for st in trange(0, ZDNA[chrm].shape - width_in, width_in):
        interval = [st, min(st + width_in, ZDNA[chrm].shape)]
        if ZDNA[chrm][interval[0]: interval[1]].any():
            all_intervals.extend(find_intervals(ZDNA[chrm][interval[0]: interval[1]]))

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 16867.17it/s]


In [83]:
if all_intervals:
    average_length = np.mean(all_intervals)
    min_length = np.min(all_intervals)
    max_length = np.max(all_intervals)
    median_length = np.median(all_intervals)

    print(f'Средний размер интервала: {average_length}')
    print(f'Минимальный размер интервала: {min_length}')
    print(f'Максимальный размер интервала: {max_length}')
    print(f'Медианный размер интервала: {median_length}')
    print(f'Количество интервалов: {len(all_intervals)}')
else:
    print('Интервалы не найдены')

Средний размер интервала: 19.700840990856804
Минимальный размер интервала: 1
Максимальный размер интервала: 1745
Медианный размер интервала: 18.0
Количество интервалов: 43639


# EndoQuad_hg19

In [6]:
width_in = 5000
all_intervals = []
for chrm in chrom_names:
    for st in trange(0, ZDNA[chrm].shape - width_in, width_in):
        interval = [st, min(st + width_in, ZDNA[chrm].shape)]
        if ZDNA[chrm][interval[0]: interval[1]].any():
            all_intervals.extend(find_intervals(ZDNA[chrm][interval[0]: interval[1]]))

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 713.32it/s]


In [7]:
if all_intervals:
    average_length = np.mean(all_intervals)
    min_length = np.min(all_intervals)
    max_length = np.max(all_intervals)
    median_length = np.median(all_intervals)

    print(f'Средний размер интервала: {average_length}')
    print(f'Минимальный размер интервала: {min_length}')
    print(f'Максимальный размер интервала: {max_length}')
    print(f'Медианный размер интервала: {median_length}')
    print(f'Количество интервалов: {len(all_intervals)}')
else:
    print('Интервалы не найдены')

Средний размер интервала: 34.31193593006106
Минимальный размер интервала: 1
Максимальный размер интервала: 134
Медианный размер интервала: 33.0
Количество интервалов: 130857


# Kouzine_hg38

In [9]:
width_in = 5000
all_intervals = []
for chrm in chrom_names:
    for st in trange(0, ZDNA[chrm].shape - width_in, width_in):
        interval = [st, min(st + width_in, ZDNA[chrm].shape)]
        if ZDNA[chrm][interval[0]: interval[1]].any():
            all_intervals.extend(find_intervals(ZDNA[chrm][interval[0]: interval[1]]))

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 13781.94it/s]


In [10]:
if all_intervals:
    average_length = np.mean(all_intervals)
    min_length = np.min(all_intervals)
    max_length = np.max(all_intervals)
    median_length = np.median(all_intervals)

    print(f'Средний размер интервала: {average_length}')
    print(f'Минимальный размер интервала: {min_length}')
    print(f'Максимальный размер интервала: {max_length}')
    print(f'Медианный размер интервала: {median_length}')
    print(f'Количество интервалов: {len(all_intervals)}')
else:
    print('Интервалы не найдены')

Средний размер интервала: 19.700840990856804
Минимальный размер интервала: 1
Максимальный размер интервала: 1745
Медианный размер интервала: 18.0
Количество интервалов: 43639
